In [ ]:
## 以下脚本用于部署 Allin1 WebUI GUI

## 网关部署

In [8]:
%%bash
sudo chown -R ec2-user:ec2-user .
docker logout public.ecr.aws

Removing login credentials for public.ecr.aws


In [ ]:
!git clone https://YOUR_GIT_PRIVATE_KEY@github.com/xieyongliang/stable-diffusion-webui-api

In [9]:
%%capture
import boto3,os
account_id = boto3.client("sts").get_caller_identity().get("Account")
region_name = boto3.session.Session().region_name
s3uri = os.getenv('s3uri')
cmd = "/bin/sh ./build_and_deploy.sh s3://%s %s stable-diffusion-webui >>bb.log" %(s3uri,region_name)
print (cmd)
os.system(cmd)

set -e
#

# This script should be run from the repo's backend directory
#
#
# Get s3uri and region from command line
s3uri=$1
region=$2

# Get reference for all important folders
backend_dir="$PWD"
project_dir="$backend_dir/.."
build_dist_dir="$backend_dir/build/codes"
source_dir="$backend_dir/src"
build_dir="$backend_dir/build/tmp"

echo "------------------------------------------------------------------------------"
echo "[Init] Clean old dist folders"
echo "------------------------------------------------------------------------------"
echo "rm -rf $build_dist_dir"
rm -rf $build_dist_dir
echo "mkdir -p $build_dist_dir"
mkdir -p $build_dist_dir

mkdir -p ${build_dir}/

echo "------------------------------------------------------------------------------"
echo "[Rebuild] all_in_one_ai_helper lambda functions"
echo "------------------------------------------------------------------------------"

echo ${source_dir}
cd ${source_dir}/all_in_one_ai_helper
rm -r ${build_dir}

mkdir -p ${buil

In [10]:
!aws s3 presign s3://$s3uri/templates/all-in-one-ai-webui-main.yaml | cut -d '?' -f 1

https://aigc-ap-northeast-1-01f1ff50.s3.amazonaws.com/templates/all-in-one-ai-webui-main.yaml


In [12]:
cfn = "$(aws s3 presign s3://$s3uri/templates/all-in-one-ai-webui-main.yaml | cut -d '?' -f 1)"

In [17]:
!aws cloudformation create-stack --stack-name allin1ai-gui \
    --template-url $cfn \
    --parameters \
    ParameterKey=WebUIDomainName,ParameterValue="" \
    ParameterKey=UserPoolDomain,ParameterValue="" \
    ParameterKey=DomainName,ParameterValue="" \
    ParameterKey=CognitoRegion,ParameterValue="" \
    ParameterKey=UserPool,ParameterValue="" \
    ParameterKey=UserPoolClient,ParameterValue="" \
    ParameterKey=Certificate,ParameterValue="" \
    ParameterKey=AvailabilityZones,ParameterValue=ap-northeast-1a\\,ap-northeast-1c\\,ap-northeast-1d\
    ParameterKey=S3Bucket,ParameterValue=$s3uri \
    ParameterKey=S3Key,ParameterValue="" \
    --capabilities CAPABILITY_NAMED_IAM

{
    "StackId": "arn:aws:cloudformation:ap-northeast-1:544592066775:stack/allin1ai-gui/320a4a70-02b7-11ee-b0bb-064d90bde3db"
}


In [22]:
!aws cloudformation describe-stacks --stack-name allin1ai-gui --output json --query 'Stacks[0].StackStatus'

"CREATE_IN_PROGRESS"


In [20]:
WebUI_URL="$(aws cloudformation describe-stacks --stack-name allin1ai-api --output text --query 'Stacks[0].Outputs[?OutputKey==`WebUIPortalUrl`].OutputValue')"

In [152]:
!echo $WebUI_URL

https://sl5h9soz0e.execute-api.eu-central-1.amazonaws.com/Prod


## UI界面

In [ ]:
%%bash
Admin_info="$(aws cloudformation describe-stacks --stack-name allin1ai-GUI --output text --query 'Stacks[0].Outputs[?OutputKey==`Administratorlogin`].OutputValue'| cut -d ":" -f 7)"

In [ ]:
#获取登录密码

In [161]:
%%bash
cd /home/ec2-user/SageMaker/Allin1-Deploy/all-in-one-ai/sagemaker/stable-diffusion-webui/tools
./secretmanager.sh $Admin_info $region_name get